<a href="https://colab.research.google.com/github/everestso/summer25/blob/main/c166s25_PlayModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
model_name = "ALE_Pong-v5-best_-10-test_epsdec150000_rs10000_sync1000.dat"
!gdown 1OqD6LLdznrDyAtLphTQnNLAtI5bbBrRx

Downloading...
From: https://drive.google.com/uc?id=1OqD6LLdznrDyAtLphTQnNLAtI5bbBrRx
To: /content/ALE_Pong-v5-best_-10-test_epsdec150000_rs10000_sync1000.dat
100% 6.75M/6.75M [00:00<00:00, 31.4MB/s]


In [69]:
# https://drive.google.com/file/d/1RWlLZyU8ZFMvt0yYKhU8W6sodSWRLZsb/view?usp=sharing
model_name = "ALE_Pong-v5-best_-20-test_epsdec150000_rs10000_sync1000.dat"
!gdown 1RWlLZyU8ZFMvt0yYKhU8W6sodSWRLZsb

Downloading...
From: https://drive.google.com/uc?id=1RWlLZyU8ZFMvt0yYKhU8W6sodSWRLZsb
To: /content/ALE_Pong-v5-best_-20-test_epsdec150000_rs10000_sync1000.dat
100% 6.75M/6.75M [00:00<00:00, 27.4MB/s]


In [2]:
!pip install gymnasium[atari,accept-rom-license]
!pip install autorom
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
!AutoROM --accept-license

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.11/dist-packages/AutoROM/roms

Existing ROMs will be overwritten.
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/adventure.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/air_raid.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/alien.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/amidar.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/assault.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/asterix.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/asteroids.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/atlantis.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/atlantis2.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/backgammon.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/bank_heist.bin
Inst

In [4]:
import ale_py
import gymnasium as gym

env = gym.make("ALE/Pong-v5", render_mode="rgb_array")
obs, _ = env.reset()
print(obs.shape)
env.close()

(210, 160, 3)


In [5]:
from dataclasses import dataclass
import argparse
import time
import numpy as np
import collections
import typing as tt

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.tensorboard.writer import SummaryWriter

In [6]:
#dqn_model
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )
        size = self.conv(torch.zeros(1, *input_shape)).size()[-1]
        self.fc = nn.Sequential(
            nn.Linear(size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
    def forward(self, x: torch.ByteTensor):
        x = x.float() / 255.0
        return self.fc(self.conv(x))

In [7]:
#wrappers

from gymnasium import spaces
from stable_baselines3.common import atari_wrappers
from stable_baselines3.common.atari_wrappers import AtariWrapper


class ImageToPyTorch(gym.ObservationWrapper):
    """
    ImageToPyTorch: Reorders image dimensions from (H, W, C) to (C, H, W)
    for compatibility with PyTorch convolutional layers.
    """
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        obs = self.observation_space
        assert isinstance(obs, gym.spaces.Box)
        assert len(obs.shape) == 3
        new_shape = (obs.shape[-1], obs.shape[0], obs.shape[1])
        self.observation_space = gym.spaces.Box(
            low=obs.low.min(), high=obs.high.max(),
            shape=new_shape, dtype=obs.dtype)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class BufferWrapper(gym.ObservationWrapper):
    """
    BufferWrapper: Maintains a rolling window of the last `n_steps` frames
    to give the agent a sense of temporal context.
    """
    def __init__(self, env, n_steps):
        super(BufferWrapper, self).__init__(env)
        obs = env.observation_space
        assert isinstance(obs, spaces.Box)
        new_obs = gym.spaces.Box(
            obs.low.repeat(n_steps, axis=0), obs.high.repeat(n_steps, axis=0),
            dtype=obs.dtype)
        self.observation_space = new_obs
        self.buffer = collections.deque(maxlen=n_steps)

    def reset(self, *, seed: tt.Optional[int] = None, options: tt.Optional[dict[str, tt.Any]] = None):
        for _ in range(self.buffer.maxlen):
            self.buffer.append(np.zeros_like(self.env.observation_space.low))
        obs, extra = self.env.reset()
        return self.observation(obs), extra

    def observation(self, observation: np.ndarray) -> np.ndarray:
        self.buffer.append(observation)
        return np.concatenate(self.buffer)


def make_eval_env(env_name, n_steps=4, render_mode="rgb_array"):
    env = gym.make(env_name, render_mode=render_mode)
    env = AtariWrapper(env, clip_reward=False, noop_max=0)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, n_steps)
    return env


In [58]:
from pathlib import Path
from gym.wrappers import RecordVideo

def make_eval_env_with_recording(env_name, model_name, video_dir="video", n_steps=4, render_mode="rgb_array"):
    # Clean prefix from filename
    model_base = Path(model_name).stem  # removes .dat
    env = make_env(env_name, n_steps=n_steps, render_mode=render_mode)
    env = RecordVideo(
        env,
        video_folder=video_dir,
        name_prefix=model_base,
        episode_trigger=lambda ep: True  # record the first episode
    )
    return env


# Run Model

In [76]:
# Base Configuration
DEFAULT_ENV_NAME = "ALE/Pong-v5"

In [77]:
## Load the Model
env_name = DEFAULT_ENV_NAME
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# env = make_env(env_name, render_mode="rgb_array")
env = make_eval_env_with_recording(env_name, model_name)
obs, _ = env.reset()
net = DQN(env.observation_space.shape, env.action_space.n)
net.load_state_dict(torch.load(model_name, map_location="cpu"))



Creating environment ALE/Pong-v5


<All keys matched successfully>

In [78]:
from gymnasium.wrappers import RecordVideo
from IPython.display import HTML
import base64
from pathlib import Path

# Create renderable environment and wrap for video recording
video_dir = "video"
env = RecordVideo(env, video_folder=video_dir, name_prefix="dqn_pong_eval", episode_trigger=lambda x: True)

obs, _ = env.reset()
total_reward = 0.0

while True:
    with torch.no_grad():
        state_v = torch.as_tensor(obs).unsqueeze(0).to(device)
        q_vals = net(state_v)
        action = int(torch.argmax(q_vals, dim=1).item())

    obs, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward
    if terminated or truncated:
        break

env.close()
print(f"✅ Episode finished. Total reward: {total_reward:.2f}")

✅ Episode finished. Total reward: -12.00


In [79]:
from IPython.display import HTML
from pathlib import Path
import base64

def show_video_by_episode(episode_number: int, model_name: str, video_dir="video"):
    """
    Displays a specific episode's video recorded using RecordVideo, using the model filename as the prefix.

    Parameters:
    - episode_number: The episode number (0, 1, ...)
    - model_name: The full .dat model filename used as name_prefix in RecordVideo
    - video_dir: Folder where video files are stored
    """
    prefix = Path(model_name).stem  # Remove .dat extension
    filename = f"{prefix}-episode-{episode_number}.mp4"
    video_path = Path(video_dir) / filename

    if not video_path.exists():
        print(f"❌ Video not found: {video_path}")
        return

    print(f"🎥 Showing episode {episode_number}: {video_path.name}")

    with open(video_path, "rb") as f:
        video_data = f.read()
    encoded = base64.b64encode(video_data).decode("ascii")

    video_html = f"""
    <video width="640" height="480" controls style="max-width: 100%;" controlslist="nodownload" allowfullscreen>
        <source src="data:video/mp4;base64,{encoded}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    <p>💡 Right-click and choose <strong>“Open video in new tab”</strong> to use fullscreen mode.</p>
    """

    return HTML(video_html)

In [80]:
show_video_by_episode(1, model_name)

🎥 Showing episode 1: ALE_Pong-v5-best_-10-test_epsdec150000_rs10000_sync1000-episode-1.mp4
